In [ ]:

pip install -q kaggle

from google.colab import files
files.upload()

mkdir ~/.kaggle

cp kaggle.json ~/.kaggle/

chmod 600 ~/.kaggle/kaggle.json

kaggle datasets list

kaggle datasets download -d grassknoted/asl-alphabet

unzip asl-alphabet.zip

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
import os
import numpy as np
import matplotlib.pyplot as plt

# ignore information messgaes from tensorflow, but we will receieve error messages
os.environ['TFF_CPP_MIN_LOG_LEVEL'] = '2'

%matplotlib inline

gpu_device = tf.config.experimental.list_physical_devices('GPU')
print(f"Number of GPU = {len(gpu_device)}")
tf.config.experimental.set_memory_growth(gpu_device[0], True)

train_dir = '/content/asl_alphabet_train/asl_alphabet_train/'
test_dir = '/content/asl_alphabet_test/asl_alphabet_test/'
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']
plt.figure(figsize=(11, 11))
for i in range (0,29):
    plt.subplot(7,7,i+1)
    plt.xticks([])
    plt.yticks([])
    path = train_dir + "/{0}/{0}1.jpg".format(classes[i])
    img = plt.imread(path)
    plt.imshow(img)
    plt.xlabel(classes[i])
    
    
from time import time
import cv2
from tensorflow.keras import utils
images = []
labels = []
size = 32,32
index = -1
for folder in os.listdir(train_dir):
    index +=1
    for image in os.listdir(train_dir + "/" + folder):
        temp_img = cv2.imread(train_dir + '/' + folder + '/' + image)
        temp_img = cv2.resize(temp_img, size)
        images.append(temp_img)
        labels.append(index)

images = np.array(images)
images = images.astype('float32')/255.0
labels = utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.1)

print(f"X_train= {x_train.shape} Y_train= {y_train.shape}")
print(f"X_test= {x_test.shape} Y_test= {x_test.shape}")

model = Sequential()
model.add(VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3)))
model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(29, activation='softmax'))
model.summary()

classes = 29
lr = 0.0001
adam = Adam(learning_rate=lr)
model.compile(
    optimizer=adam, 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

h = model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1, shuffle = True, verbose=1)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(h.history['accuracy'], '--', label='train accuracy')
plt.plot(h.history['val_accuracy'], '--', label = 'validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend(loc='lower right')

plt.subplot(1,2,2)
plt.plot(h.history['loss'], '--', label='train loss')
plt.plot(h.history['val_loss'], '--', label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend(loc='upper right')

plt.show()

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)
